In [4]:
import pystan
import seaborn as sns
import numpy as np
import pandas as pd

In [5]:
sns.set(rc={'figure.figsize':(14, 10)})

## 16.4

In [6]:
X = [4, 18, 6, 4, 5, 6, 4, 6, 16, 7]
assert len(X) == 10

### 16.4.1

$ \theta_b \sim \mathcal{U}(0,1) $

$ \theta_m \sim \mathcal{U}(0,1) $

$ s_i \sim \mathcal{B}(0.5) $

$ X[i] \sim \mathcal{B}(20, \theta_m) \;\;\;{if}\; s_i = 1 $

$ X[i] \sim \mathcal{B}(20, \theta_b) \;\;\; {if}\;s_i = 0 $

$s_i = 1$ means malignant, $0$ - benign

### 16.4.2

s == 1 -> benign, s == 2 -> malignant

In [7]:
'''
transformed parameters {
    matrix[nStudy, 2] logp;
    for(i in 1:N){
        logp[i, 1] = log(0.5) + binomial_lpmf(X[i] | N, theta[1]);
        logp[i, 2] = log(0.5) + binomial_lpmf(X[i] | N, theta[2]);
    }
}
'''

'\ntransformed parameters {\n    matrix[nStudy, 2] logp;\n    for(i in 1:N){\n        matrix[i, 1] = log(0.5) + binomial_lpmf(X[i] | N, theta[1]);\n        matrix[i, 2] = log(0.5) + binomial_lpmf(X[i] | N, theta[2]);\n    }\n}\n'

### 16.4.3 

In [27]:
ocode = """
data {
    int<lower=1> N;
    int<lower=0, upper=20> X[N];

}

parameters {
    ordered[2] alpha;
}

transformed parameters {
    real<lower=0, upper=1> theta[2];
    matrix[N, 2] logp;
    
    theta[1] = inv_logit(alpha[1]);
    theta[2] = inv_logit(alpha[2]);
    
    for(i in 1:N){
        logp[i, 1] = log(0.5) + binomial_logit_lpmf(X[i] | 20, alpha[1]);
        logp[i, 2] = log(0.5) + binomial_logit_lpmf(X[i] | 20, alpha[2]);
    }
}
model {
   for(i in 1:N){
        target += log_sum_exp(logp[i]);
    }
}


"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_50259110c51b639de2a7299ad999d3ae NOW.


In [28]:
data = {'N': len(X), 'X': X}

In [29]:
fit = sm.sampling(data=data, iter=1000, chains=4, seed=1)
fit

Inference for Stan model: anon_model_50259110c51b639de2a7299ad999d3ae.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[1]    -1.04  4.9e-3   0.18  -1.38  -1.16  -1.03  -0.92  -0.69   1320    1.0
alpha[2]      1.8    0.01   0.47   0.97   1.47   1.76    2.1   2.84   1168    1.0
theta[1]     0.26  9.6e-4   0.03    0.2   0.24   0.26   0.28   0.34   1296    1.0
theta[2]     0.85  1.7e-3   0.06   0.72   0.81   0.85   0.89   0.94   1167    1.0
logp[1,1]   -2.48  6.9e-3   0.24  -3.11   -2.6  -2.43   -2.3  -2.22   1214    1.0
logp[2,1]  -20.25    0.06   2.28  -24.8 -21.75 -20.15 -18.71 -15.94   1321    1.0
logp[3,1]   -2.48  4.5e-3   0.16   -2.9  -2.54  -2.42  -2.37  -2.35   1256    1.0
logp[4,1]   -2.48  6.9e-3   0.24  -3.11   -2.6  -2.43   -2.3  -2.22   1214    1.0
logp[5,1]   -2.36  3.0e-3    0.1  -2.64  -2.38  -2.32   -2.3  -2.29   1052 

### 16.4.4

In [30]:
ocode = """
data {
    int<lower=1> N;
    int<lower=0, upper=20> X[N];

}

parameters {
    ordered[2] alpha;
}

transformed parameters {
    real<lower=0, upper=1> theta[2];
    matrix[N, 2] logp;
    
    theta[1] = inv_logit(alpha[1]);
    theta[2] = inv_logit(alpha[2]);
    
    for(i in 1:N){
        logp[i, 1] = log(0.5) + binomial_logit_lpmf(X[i] | 20, alpha[1]);
        logp[i, 2] = log(0.5) + binomial_logit_lpmf(X[i] | 20, alpha[2]);
    }
}
model {
   for(i in 1:N){
        target += log_sum_exp(logp[i]);
    }
}

generated quantities {
    real pState[N];
    for(i in 1:N){
        pState[i] = exp(logp[i,1]) / (exp(logp[i,1]) + exp(logp[i,2]));
    }
}



"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8787560796fec25e7a4422be212dbc6b NOW.


In [31]:
data = {'N': len(X), 'X': X}

In [32]:
fit = sm.sampling(data=data, iter=1000, chains=4, seed=1)
fit

Inference for Stan model: anon_model_8787560796fec25e7a4422be212dbc6b.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd    2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[1]    -1.04  4.9e-3   0.18   -1.38  -1.16  -1.03  -0.92  -0.69   1320    1.0
alpha[2]      1.8    0.01   0.47    0.97   1.47   1.76    2.1   2.84   1168    1.0
theta[1]     0.26  9.6e-4   0.03     0.2   0.24   0.26   0.28   0.34   1296    1.0
theta[2]     0.85  1.7e-3   0.06    0.72   0.81   0.85   0.89   0.94   1167    1.0
logp[1,1]   -2.48  6.9e-3   0.24   -3.11   -2.6  -2.43   -2.3  -2.22   1214    1.0
logp[2,1]  -20.25    0.06   2.28   -24.8 -21.75 -20.15 -18.71 -15.94   1321    1.0
logp[3,1]   -2.48  4.5e-3   0.16    -2.9  -2.54  -2.42  -2.37  -2.35   1256    1.0
logp[4,1]   -2.48  6.9e-3   0.24   -3.11   -2.6  -2.43   -2.3  -2.22   1214    1.0
logp[5,1]   -2.36  3.0e-3    0.1   -2.64  -2.38  -2.32   -2.3  -2.

In [33]:
X

[4, 18, 6, 4, 5, 6, 4, 6, 16, 7]

### 16.4.5 - 16.4.8 todo